In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm
import re
from sklearn.preprocessing import LabelEncoder
import pickle
import os
import torch
import platform
from sentence_transformers import SentenceTransformer

In [6]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")  
solution_df = pd.read_csv("data/solution.csv")

In [7]:
if torch.cuda.is_available():
    device = "cuda"
elif platform.system() == "Darwin" and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print("Using device:", device)

Using device: cuda


In [8]:
model = SentenceTransformer("intfloat/e5-large-v2", device=device)

train_texts = train_df["body"].tolist()
train_labels = train_df["rule_violation"].astype(int).tolist()

pos_descriptions = [
    "This text violates the rule.",
    "This text does not violate the rule."
]

In [9]:
triplets = []

for text, label in zip(train_texts, train_labels):
    anchor = text
    positive = pos_descriptions[label]
    negative = pos_descriptions[1 - label]

    triplets.append(
        InputExample(
            texts=[anchor, positive, negative]
        )
    )

print(f"Built {len(triplets)} triplets.")


Built 2029 triplets.


In [11]:
train_loss = losses.TripletLoss(model, distance_metric=losses.TripletDistanceMetric.EUCLIDEAN, triplet_margin=1.0)
train_loader = DataLoader(triplets, shuffle=True, batch_size=16)

model.fit(
    train_objectives=[(train_loader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True,
    output_path="./e5-large-v2-triplet"
)
print("Triplet fine-tuning complete.")
print("Model saved to ./e5-large-v2-triplet")


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


Triplet fine-tuning complete.
Model saved to ./e5-large-v2-triplet
